# Creating and filling a consolidated table of M-ligue results

### Create Consolidated table for all tournaments

Execute this step only the first time. This code creates empty file with specific column headers, and ongoing tournaments result are going to be appended to this file.

In [237]:
import pandas as pd

Consolidated = pd.ExcelWriter('Consolidated.xlsx')
df = pd.DataFrame(columns = ['Rank', 'Team', 'City', 'ID', 'Captain', 'Location', 'Status', 'Total', 'Time','Question','Result','Tournament'])
df.to_excel('Consolidated.xlsx',index=0)

### Prepare Consolidated table

Start from this step if you already have a Consolidated table file.

Consolidated table should be opened and ready when we will append manipulated data in it.

In [12]:
import pandas as pd

df = pd.read_excel('Consolidated.xlsx')

### Download the results of the particular game

After the game finished the file with results is ready to download from the chgk-world.ru page.

In [3]:
import urllib.request

#Find and replace old Excel file name with a new one using 'F' command.
destination = r"C:\Users\Lenovo\3 Work\Data Visualisation\Projects\Son of mom's friend\Games results\2022 July Final2.xlsx"

# Insert the link to the game results you want to download.
url = 'https://chgk-world.ru/tournaments/1166/table/simple/verbose.xlsx'

urllib.request.urlretrieve(url, destination)

('2022 July Final2.xlsx', <http.client.HTTPMessage at 0x28e50cb1a60>)

### Change '+' and '-' to '1' and '0' as a description of right and wrong answers

Correct and incorrect answers are depicted as '+'s and '-'s. To be able to manipulate and aggregate them as numbers let's change them to 1 and 0.

In [4]:
from pathlib import Path
import openpyxl

replacement_pair = {"+": "1", "-": "0", "?": "0", "": "0"}

wb = openpyxl.load_workbook(r"C:\Users\Lenovo\3 Work\Data Visualisation\Projects\Son of mom's friend\Games results\2022 July Final2.xlsx")

for ws in wb.worksheets:
    for row in ws.iter_rows():
        for cell in row:
            if cell.value in replacement_pair.keys():
                cell.value = replacement_pair.get(cell.value)
                
wb.save(r"C:\Users\Lenovo\3 Work\Data Visualisation\Projects\Son of mom's friend\Games results\2022 July Final2.xlsx")

### Pivoting number of questions from columns to values

In the initial file we downloaded from the web-page every question's result shown in a separate column. We want to transform the table so we had one column for question's number and one column for result of the particular team on this question (1 or 0).

In [5]:
wb = pd.read_excel(r"C:\Users\Lenovo\3 Work\Data Visualisation\Projects\Son of mom's friend\Games results\2022 July Final2.xlsx")

wb = wb.melt(id_vars = ["#","Название","Город", "Makid", "Капитан", "Площадка", "Зачет", "Сумма", "Время+"], 
            var_name = "Question")   

wb.to_excel(r"C:\Users\Lenovo\3 Work\Data Visualisation\Projects\Son of mom's friend\Games results\2022 July Final2.xlsx",index=0)

### Rename columns

Change column names to the ones how they will be shown in further visualization.

In [6]:
wb.rename(columns = {'#':'Rank', 'Название':'Team', 'Город':'City', 'Makid':'ID', 'Капитан':'Captain', 'Площадка':'Location', 
                     'Зачет':'Status', 'Сумма':'Total', 'Время+':'Time', 'value':'Result'}, inplace = True)

wb.to_excel(r"C:\Users\Lenovo\3 Work\Data Visualisation\Projects\Son of mom's friend\Games results\2022 July Final2.xlsx",index=0)

### Add file name to column

Since we are going to add the game results after every game, we want to understand what game it was. There is no information about the game in the initial file, so we want to add game name to our file.

In [7]:
wb['Tournament'] = '2022 July Final2'

wb.to_excel(r"C:\Users\Lenovo\3 Work\Data Visualisation\Projects\Son of mom's friend\Games results\2022 July Final2.xlsx",index=0)

### Append last tournament to Consolidated file

After we finished manipulating downloaded file, let's add it to our database.

In [14]:
pd.concat([df,wb]).to_excel('Consolidated.xlsx',index=0)